In [1]:
import os
import platform
import sys
import shutil
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import torch
import torchvision

In [2]:
#电脑配置信息
print("Operating system: ", platform.platform(), platform.version())
print("Python version", sys.version)
print("Pandas version: ", pd.__version__)
print("Sklearn version: ", sklearn.__version__)
print("Pytorch version: ", torch.__version__)

Operating system:  Windows-10-10.0.22621-SP0 10.0.22621
Python version 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)]
Pandas version:  2.0.3
Sklearn version:  1.3.2
Pytorch version:  2.1.1+cpu


In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print('Using device:', device)

Using device: cpu


In [4]:
RANDOM_STATE = 0
TRAIN_VS_VALIDATION_SET_PERCENTANGE = 0.8
BATCH_SIZE = 32
TRAIN_FULL_IMAGES_PATH = './imageclassification/temp/train'
TEST_FULL_IMAGES_PATH = './imageclassification/temp/test'
TRAIN_IMAGES_REORGANIZED_PATH = './imageclassification/temp/train_reorganized'
VALID_IMAGES_REORGANIZED_PATH = './imageclassification/temp/valid_reorganized'
TEST_IMAGES_REORGANIZED_PATH = './imageclassification/temp/test_reorganized'
TEST_IMAGES_REORGANIZED_PLACEHOLDER_LABEL_PATH = f"{TEST_IMAGES_REORGANIZED_PATH}/unknown"

In [5]:
train_full_labels = pd.read_csv('./imageclassification/temp/trainLabels.csv', index_col='id')

In [6]:
from sklearn.model_selection import train_test_split

train_labels, valid_labels = train_test_split(
    train_full_labels,
    train_size=TRAIN_VS_VALIDATION_SET_PERCENTANGE,
    shuffle=True,
    random_state=RANDOM_STATE
)

In [7]:
train_labels

,label
id,
20331,cat
17533,horse
45820,dog
34808,horse
31889,airplane
...,...
21244,airplane
45892,dog
42614,dog


In [8]:
#重新组织图像文件
def reorganize_images(labels, source_path, destination_path):
    for label in labels["label"].unique():
        class_path = os.path.join(destination_path, str(label))
        print(f"Creating class path: {class_path}")
        os.makedirs(class_path, exist_ok=True)
        
    for idx, row in labels.iterrows():
        id = row.name
        label = row["label"]
        source_image_path = os.path.join(source_path, f"{id}.png")
        destination_image_path = os.path.join(destination_path, str(label), f"{id}.png")

        # 如果目标图像文件已经存在，则删除
        if os.path.isfile(destination_image_path):
            os.remove(destination_image_path)

        # 复制文件到新目录
        shutil.copy(source_image_path, destination_image_path)

# 使用示例
reorganize_images(train_labels, TRAIN_FULL_IMAGES_PATH, TRAIN_IMAGES_REORGANIZED_PATH)
reorganize_images(valid_labels, TRAIN_FULL_IMAGES_PATH, VALID_IMAGES_REORGANIZED_PATH)

Creating class path: ./imageclassification/temp/train_reorganized\cat
Creating class path: ./imageclassification/temp/train_reorganized\horse
Creating class path: ./imageclassification/temp/train_reorganized\dog
Creating class path: ./imageclassification/temp/train_reorganized\airplane
Creating class path: ./imageclassification/temp/train_reorganized\ship
Creating class path: ./imageclassification/temp/train_reorganized\truck
Creating class path: ./imageclassification/temp/train_reorganized\bird
Creating class path: ./imageclassification/temp/train_reorganized\automobile
Creating class path: ./imageclassification/temp/train_reorganized\frog
Creating class path: ./imageclassification/temp/train_reorganized\deer
Creating class path: ./imageclassification/temp/valid_reorganized\horse
Creating class path: ./imageclassification/temp/valid_reorganized\dog
Creating class path: ./imageclassification/temp/valid_reorganized\frog
Creating class path: ./imageclassification/temp/valid_reorganized\d

In [9]:
from torchvision.transforms import Compose, ToTensor, Normalize

mean = torch.tensor([0.4914, 0.4822, 0.4465])
std = torch.tensor([0.2023, 0.1994, 0.2010])

transform = Compose([
    ToTensor(),
    Normalize(mean, std)
])

In [10]:
from torchvision.datasets import ImageFolder

train_images = ImageFolder(TRAIN_IMAGES_REORGANIZED_PATH, transform=transform)
valid_images = ImageFolder(VALID_IMAGES_REORGANIZED_PATH, transform=transform)

In [11]:
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.downsample(identity)
        out = self.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self.make_layer(block, 64, layers[0])
        self.layer2 = self.make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self.make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self.make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


net = ResNet(BasicBlock, [2, 2, 2, 2])


In [12]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [13]:
from torch.utils.data import DataLoader

# Create train and validation batch for training
train_loader = DataLoader(dataset=train_images, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
valid_loader = DataLoader(dataset=valid_images, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

In [14]:
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for batch_index, batch in enumerate(train_loader):
        #获取输入
        sample_images, sample_labels = batch

        # 参数梯度归零
        # 默认情况下，pytorch在反向传递中会累积梯度
        optimizer.zero_grad()

        # forward + backward + optimize
        predicted_labels = net(sample_images)
        loss = criterion(predicted_labels, sample_labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        # 每五百个小批量打印一次
        if batch_index % 500 == 0:
            print(f"epoch {epoch} batch {batch_index} loss {running_loss / 2000}")
        running_loss = 0

    loss = criterion(predicted_labels, sample_labels)
    train_loss = loss.item()

epoch 0 batch 0 loss 0.0012187812328338624
epoch 0 batch 500 loss 0.0006368532776832581
epoch 0 batch 1000 loss 0.0005444280505180358
epoch 1 batch 0 loss 0.0006210973262786865
epoch 1 batch 500 loss 0.0003575623035430908
epoch 1 batch 1000 loss 0.000649368405342102
epoch 2 batch 0 loss 0.0004910331964492798
epoch 2 batch 500 loss 0.0005058587789535522
epoch 2 batch 1000 loss 0.0004096373617649078
epoch 3 batch 0 loss 0.00044260162115097043
epoch 3 batch 500 loss 0.0004413345158100128
epoch 3 batch 1000 loss 0.00038347592949867246
epoch 4 batch 0 loss 0.00023006980121135712
epoch 4 batch 500 loss 0.00023925028741359711
epoch 4 batch 1000 loss 0.00042264413833618166
epoch 5 batch 0 loss 0.00028389677405357363
epoch 5 batch 500 loss 0.00032715144753456114
epoch 5 batch 1000 loss 0.0002906356155872345
epoch 6 batch 0 loss 0.00022626714408397676
epoch 6 batch 500 loss 0.00021137554943561555
epoch 6 batch 1000 loss 0.0002653398215770721
epoch 7 batch 0 loss 8.055152744054794e-05
epoch 7 bat

In [15]:
# 创建测试集图像目录
os.makedirs(TEST_IMAGES_REORGANIZED_PATH, exist_ok=True)

# 检查是否已经存在链接
if os.path.islink(TEST_IMAGES_REORGANIZED_PLACEHOLDER_LABEL_PATH):
    print("Test set has been already linked")
else:
    # 创建符号链接
    os.system(f"mklink /D {TEST_IMAGES_REORGANIZED_PLACEHOLDER_LABEL_PATH} {TEST_FULL_IMAGES_PATH}")
    print("Test set linked successfully")

Test set linked successfully


In [16]:
test_images = ImageFolder(TEST_IMAGES_REORGANIZED_PATH, transform=transform)

In [17]:
test_loader = DataLoader(dataset=test_images, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

preds = []
for X, _ in test_loader:
    y_hat = net(X)
    preds.extend(y_hat.argmax(dim=1))
# 从测试集图像路径中提取图像的 id（去掉路径前缀和文件扩展名）
# 从测试集图像路径中提取图像的 id（去掉路径前缀和文件扩展名）
ids = [os.path.splitext(os.path.basename(path))[0] for path, _ in test_loader.dataset.imgs]

# 创建一个DataFrame，其中包含图像的id和对应的预测标签
df = pd.DataFrame({'id': ids, 'label': map(lambda i: train_images.classes[i], preds)})

# 将DataFrame保存到CSV文件中
df.to_csv('./imageclassification/working/submission.csv', index=False)

In [18]:
import pandas as pd

# 读取正确标签的CSV文件，假设文件名为 'true_labels.csv'
true_labels = pd.read_csv('./imageclassification/temp/testLables.csv', index_col='id')

# 读取预测标签的CSV文件，假设文件名为 'predicted_labels.csv'
predicted_labels = pd.read_csv('./imageclassification/working/submission.csv', index_col='id')

# 将两个DataFrame按照id进行合并
merged_data = pd.merge(true_labels, predicted_labels, left_index=True, right_index=True, suffixes=('_true', '_predicted'))

# 统计正确的数量
correct_count = (merged_data['label_true'] == merged_data['label_predicted']).sum()

# 打印结果
print(f"Number of correct predictions: {correct_count}")
accury = correct_count/10000
print(accury)

Number of correct predictions: 7088
0.7088
